In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

bronze_client=spark.table("bronze_client")
w_client=Window.partitionBy("CLIENT_ID").orderBy(F.desc("ROW_UPDATE_DATE"))
silver_client=(
    bronze_client
    .withColumn("rn",F.row_number().over(w_client))
    .filter("rn=1")
    .drop('rn')
    )

SILVER_TABLE="silver_client"
#change all the date column to date type
silver_client=(
    silver_client
    .withColumn("CLIENT_SERVICE_START_DATE",F.to_date("CLIENT_SERVICE_START_DATE"))
    .withColumn("ROW_INSERT_DATE",F.to_date("ROW_INSERT_DATE"))
    .withColumn("ROW_UPDATE_DATE",F.to_date("ROW_UPDATE_DATE"))
)

# if silver table does not exist, create one
if not spark.catalog.tableExists(SILVER_TABLE):
    (silver_client.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(SILVER_TABLE)
    )

#Merge:idempotent upsert
silver_client.createOrReplaceTempView("silver_upsert")

spark.sql(f"""
MERGE INTO {SILVER_TABLE} t
USING silver_upsert s
ON t.CLIENT_ID=s.CLIENT_ID
WHEN MATCHED AND (
    coalesce(s.ROW_UPDATE_DATE,s.ROW_INSERT_DATE)>coalesce(t.ROW_UPDATE_DATE,t.ROW_INSERT_DATE)
) THEN UPDATE SET *
WHEN NOT MATCHED THEN
INSERT *
""")  
#3)Check
spark.sql(f"SELECT COUNT(*) cnt FROM {SILVER_TABLE}").display()
spark.sql(f"SELECT * FROM {SILVER_TABLE} ORDER BY ROW_UPDATE_DATE DESC").display()